In [ ]:
# %%
import pandas as pd
import numpy as np
from sklearn import linear_model
import requests
from nba_api.stats import endpoints
from nba_api.stats.static import players
from nba_api.stats.endpoints import shotchartdetail
from matplotlib import pyplot as plt
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playergamelog
import datetime
from retrying import retry
import requests.exceptions




In [ ]:
# Define a retry decorator with exponential backoff strategy
@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000)
def get_game_logs(player_id):
    try:
        game_log = playergamelog.PlayerGameLog(player_id=player_id)
        game_log_data = game_log.get_data_frames()[0]
        return game_log_data
    except requests.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        raise

# List of past player IDs or names
past_players = [
    {'id': 76001, 'name': 'Michael Jordan'},
    {'id': 977, 'name': 'Kobe Bryant'},
    # Add more past players as needed
]

# Iterate through each past player
for player_info in past_players:
    player_id = player_info['id']
    player_name = player_info['name']
    
    try:
        # Get player's game logs with retry wrapper
        game_log_data = get_game_logs(player_id)
        
        # Process game log data
        max_points = game_log_data['PTS'].max()
        max_rebounds = game_log_data['REB'].max()
        max_assists = game_log_data['AST'].max()
        
        # Print specific records
        print(f"Player: {player_name}")
        print(f"Max points in a single game: {max_points}")
        print(f"Max rebounds in a single game: {max_rebounds}")
        print(f"Max assists in a single game: {max_assists}")
        print("\n")
    except Exception as e:
        print(f"Error occurred while retrieving data for {player_name}: {e}")

get_game_logs(977)

In [ ]:
# %%
@retry(wait_exponential_multiplier=1000, wait_exponential_max=10000)
def get_player_log(player_id):
    # Get player's game logs
    try:
        # Get player's game logs
        game_log = playergamelog.PlayerGameLog(player_id=player_id)
        game_log_data = game_log.get_data_frames()[0]
        return game_log_data[['GAME_DATE', 'PTS', 'REB', 'AST', 'STL', 'BLK']]
    except requests.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        raise

# Get all active players
all_players = players.get_active_players()
playersToMaxScore = dict()
# Iterate through each player
for player in all_players:
    player_id = player['id']
    player_name = player['full_name']
    
    try:

        # Get headline stats
        headline_stats = get_player_log(player_id)
        playersToMaxScore[player_name] = headline_stats['PTS'].max()
    except Exception as e:
        print(f"Error occurred while retrieving data for {player_name}: {e}")

playersToMaxScore



In [ ]:
# List of past player IDs or names
past_players = [
    {'id': 76001, 'name': 'Michael Jordan'},
    {'id': 977, 'name': 'Kobe Bryant'},
    # Add more past players as needed
]

def get_max_details(player_list):
    # Iterate through each past player
    player_max_data = dict()
    for player_info in player_list:
        player_id = player_info['id']
        player_name = player_info['name']
        player_max_stats = dict()
        try:
            # Get player's game logs
            game_log = playergamelog.PlayerGameLog(player_id=player_id)
            game_log_data = game_log.get_data_frames()[0]
            cols = ['PTS','REB','AST','STL','BLK']
            # Get maximum stats
            player_max_stats['max_points'] = game_log_data['PTS'].max()
            player_max_stats['max_rebounds'] = game_log_data['REB'].max()
            player_max_stats['max_assists'] = game_log_data['AST'].max()
            player_max_stats['max_steals'] = game_log_data['STL'].max()
            player_max_stats['max_blocks'] = game_log_data['BLK'].max()
            print("\n")
        except Exception as e:
            print(f"Error occurred while retrieving data for {player_name}: {e}")

        player_max_data[player_name] = player_max_stats
    return player_max_data

In [ ]:
# Get all active players
all_players = players.get_active_players()

# Iterate through each player
for player in all_players:
    player_id = player['id']
    player_name = player['full_name']
    
    # Get player's game logs
    game_log = playergamelog.PlayerGameLog(player_id=player_id)
    game_log_data = game_log.get_data_frames()[0]
    
    # Get headline stats
    headline_stats = game_log_data[['GAME_DATE', 'PTS', 'REB', 'AST', 'STL', 'BLK']]
    
    # Print headline stats
    print(f"Headline stats for {player_name}:")
    print(headline_stats)
    print("\n")